<a href="https://colab.research.google.com/github/Prishita17/Optimised_Movie_Recommendation_system/blob/main/Optimised_Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Dependencies


In [1]:
!pip install surprise pandas scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-linux_x86_64.whl size=2611311 sha256=1bdd27b10557fa23b8122e69de06210c428e5048f227b98013a4ba13a56cf3d0
  Stored in directory: /root/.cache/pip/wheels/75/fa/bc/739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise


Import Libraries

In [12]:
import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

Install numpy


In [2]:
!pip install "numpy<2"

Load MovieLens 100k dataset

In [3]:
ratings = pd.read_csv("u.data", sep="\t", names=["user_id", "movie_id", "rating", "timestamp"])
movies = pd.read_csv("u.item", sep="|", names=range(24), encoding="latin-1")
movies = movies[[0, 1, 2, 3, 4, 5]]  # keep useful columns
movies.columns = ["movie_id", "title", "release_date", "video_release", "IMDb_URL", "unknown"]


FileNotFoundError: [Errno 2] No such file or directory: 'u.data'

Download Dataset

In [6]:
ratings = pd.read_csv("ml-100k/u.data", sep="\t", names=["user_id", "movie_id", "rating", "timestamp"])
movies = pd.read_csv("ml-100k/u.item", sep="|", names=range(24), encoding="latin-1")
movies = movies[[0, 1, 2, 3, 4, 5]]  # keep useful columns
movies.columns = ["movie_id", "title", "release_date", "video_release", "IMDb_URL", "unknown"]

Dataset Downloading...

In [4]:
import urllib.request
import os

# Download the dataset if it doesn't exist
if not os.path.exists('ml-100k'):
    print('Downloading dataset...')
    urllib.request.urlretrieve('http://files.grouplens.org/datasets/movielens/ml-100k.zip', 'ml-100k.zip')
    !unzip ml-100k.zip
    print('Dataset downloaded and extracted.')
else:
    print('Dataset already exists.')

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  inflating: ml-100k/ub.base         
  inflating: ml-100k/ub.test         
Dataset downloaded and extracted.


Collaborative Filtering (SVD)

In [8]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[["user_id", "movie_id", "rating"]], reader)
trainset, testset = train_test_split(data, test_size=0.2)

model = SVD()
model.fit(trainset)

In [9]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[["user_id", "movie_id", "rating"]], reader)
trainset, testset = train_test_split(data, test_size=0.2)

model = SVD()
model.fit(trainset)

Predict for one user

In [10]:
user_id = 196  # sample user
user_movies = ratings[ratings["user_id"] == user_id]["movie_id"].tolist()

all_movies = movies["movie_id"].tolist()
predictions = [(movie, model.predict(user_id, movie).est) for movie in all_movies if movie not in user_movies]
top5 = sorted(predictions, key=lambda x: x[1], reverse=True)[:5]

print("Top-5 Recommended Movies (Collaborative Filtering):")
for movie_id, score in top5:
    print(movies[movies["movie_id"] == movie_id]["title"].values[0], " (score:", round(score, 2), ")")


Top-5 Recommended Movies (Collaborative Filtering):
Shawshank Redemption, The (1994)  (score: 4.55 )
Rear Window (1954)  (score: 4.5 )
Schindler's List (1993)  (score: 4.5 )
Henry V (1989)  (score: 4.45 )
Close Shave, A (1995)  (score: 4.4 )


Content-Based Filtering (using genres)

In [11]:
genres = movies.iloc[:, 5:]  # one-hot encoded genres (in full dataset)
cosine_sim = cosine_similarity(genres, genres)

movie_index = 50  # pick any movie
similar_movies = list(enumerate(cosine_sim[movie_index]))
similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)[:5]

print("\nMovies similar to:", movies.iloc[movie_index]["title"])
for idx, sim in similar_movies:
    print(movies.iloc[idx]["title"])


Movies similar to: Legends of the Fall (1994)
Toy Story (1995)
GoldenEye (1995)
Four Rooms (1995)
Get Shorty (1995)
Copycat (1995)
